In [20]:
# 这里我们实现了ResNet 18，原论文中还讨论了更深的配置。尝试实现它们。（提示：参考论文中的表1）
# http://zh.gluon.ai/chapter_convolutional-neural-networks/resnet-gluon.html
# 尝试ResNet 50

from mxnet.gluon import nn
from mxnet import nd

class ResidualBottleneck(nn.Block):
#     def __init__(self, channels_1, channels_3, **kwargs):
#         super(ResidualBottleneck, self).__init__(**kwargs)
# #         strides = 1 if same_shape else 2
# #         self.strides = 2
#         self.conv1 = nn.Conv2D(channels_1, kernel_size=1,
#                               strides=1)
#         self.bn1 = nn.BatchNorm()
#         self.conv2 = nn.Conv2D(channels_1, kernel_size=3, padding=1)
#         self.bn2 = nn.BatchNorm()
#         self.conv3 = nn.Conv2D(channels_3, kernel_size=1,
#                                   strides=2)

    def __init__(self, channels_in, channels_out, same_shape=False, **kwargs):
        super(ResidualBottleneck, self).__init__(**kwargs)
        self.same_shape = same_shape
        strides = 1 if same_shape else 2
        self.conv1 = nn.Conv2D(channels_in, kernel_size=1,
                              strides=strides)
        self.bn1 = nn.BatchNorm()
        self.conv2 = nn.Conv2D(channels_in, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm()
        self.conv3 = nn.Conv2D(channels_out, kernel_size=1,
                              strides=strides)
        self.bn3 = nn.BatchNorm()
        if not same_shape:
            self.conv4 = nn.Conv2D(channels_out, kernel_size=1,
                                  strides=strides)
    
#     def forward(self, x):
#         print(x.shape)
#         out = nd.relu(self.bn1(self.conv1(x)))
#         print(out.shape)
#         out = nd.relu(self.bn2(self.conv2(out)))
#         print(out.shape)
#         out = self.conv3(out)
#         print(out.shape)
#         return nd.relu(out + x)
    def forward(self, x):
        print('x.shape:', x.shape)
        out = nd.relu(self.bn1(self.conv1(x)))
        print(out.shape)
        out = nd.relu(self.bn2(self.conv2(out)))
        print(out.shape)
        out = self.bn3(self.conv3(out))
        print(out.shape)
        if not self.same_shape:
            x = self.conv4(x)
            print('x.shape:',x.shape)
        return nd.relu(out + x)

# 构建ResNet
class ResNet50(nn.Block):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super(ResNet50, self).__init__(**kwargs)
        self.verbose = verbose
        # add name_scope on the outermost Sequential
        with self.name_scope():
            # block 1
            b1 = nn.Conv2D(64, kernel_size=7, strides=2)
            # block 2
            b2 = nn.Sequential()
            b2.add(
                nn.MaxPool2D(pool_size=3, strides=2),
                ResidualBottleneck(64, 256),
                ResidualBottleneck(64, 256),
                ResidualBottleneck(64, 256)
            )
            # block 3
            b3 = nn.Sequential()
            b3.add(
                ResidualBottleneck(128, 512),
                ResidualBottleneck(128, 512),
                ResidualBottleneck(128, 512),
                ResidualBottleneck(128, 512)
            )
            # block 4
            b4 = nn.Sequential()
            b4.add(
                ResidualBottleneck(256, 1024),
                ResidualBottleneck(256, 1024),
                ResidualBottleneck(256, 1024),
                ResidualBottleneck(256, 1024),
                ResidualBottleneck(256, 1024),
                ResidualBottleneck(256, 1024)
            )
            # block 5
            b5 = nn.Sequential()
            b5.add(
                ResidualBottleneck(512, 2048),
                ResidualBottleneck(512, 2048),
                ResidualBottleneck(512, 2048)
            )
            # block 6
            b6 = nn.Sequential()
            b6.add(
                nn.AvgPool2D(pool_size=3),
                nn.Dense(num_classes)
            )
            # chain all blocks together
            self.net = nn.Sequential()
            self.net.add(b1, b2, b3, b4, b5, b6)
    
    def forward(self, x):
        out = x
        for i, b in enumerate(self.net):
            out = b(out)
            if self.verbose:
                print('Block %d output: %s' % (i+1, out.shape))
        return out

In [21]:
x = nd.random.uniform(shape=(4, 3, 96, 96))
net = ResNet50(num_classes=10, verbose=True)
# print(net)
net.initialize()
y = net(x)
print(y.shape)
print(net)

Block 1 output: (4, 64, 45, 45)
x.shape: (4, 64, 22, 22)
(4, 64, 22, 22)
(4, 64, 22, 22)
(4, 256, 22, 22)


MXNetError: [23:53:21] src/operator/tensor/./elemwise_binary_broadcast_op.h:68: Check failed: l == 1 || r == 1 operands could not be broadcast together with shapes [4,256,22,22] [4,64,22,22]

Stack trace returned 10 entries:
[bt] (0) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x3032da) [0x7f0e707072da]
[bt] (1) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x303901) [0x7f0e70707901]
[bt] (2) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0xba6477) [0x7f0e70faa477]
[bt] (3) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x24e05ba) [0x7f0e728e45ba]
[bt] (4) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x24ea589) [0x7f0e728ee589]
[bt] (5) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x241c2a9) [0x7f0e728202a9]
[bt] (6) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/